In [1]:
import os
import netCDF4
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature

In [2]:
## Load Data
#Path
os.chdir('D:\\Bigfiles\\ERA5_PCP')
filename_PCP1 = 'ERA5_daily_pcp_1979-2020.nc'

#Load netcdf
ncdf = netCDF4.Dataset(filename_PCP1)
print(ncdf.variables)
print(ncdf.dimensions)

#Get dimension
lons = ncdf.variables['lon'][:]
lats = ncdf.variables['lat'][:]
times = ncdf.variables['time'][:]

#Get variables
pcp = ncdf.variables['pcp_Accum'][:]

#Calculate datetime
dtimes = np.empty(len(times), dtype='datetime64[h]')
for i,t in enumerate(times):
    dtimes[i] = np.datetime64('1900-01-01') + np.timedelta64(np.int64(t),'h')
dtimes = dtimes.astype('datetime64')

<ipython-input-2-edfcd4eb1f15>:8: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  print(ncdf.variables)


{'time': <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    units: days since 1950-01-01 00:00:00.0
unlimited dimensions: 
current shape = (14973,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'lon': <class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
unlimited dimensions: 
current shape = (1440,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'lat': <class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
unlimited dimensions: 
current shape = (121,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'pcp_Accum': <class 'netCDF4._netCDF4.Variable'>
float32 pcp_Accum(time, lat, lon)
    units: m
unlimited dimensions: 
current shape = (14973, 121, 1440)
filling on, default _FillValue of 9.969209968386869e+36 used}
{'time': <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 14973, 'lon': <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 1440, 'lat': <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 121}


In [3]:
print(dtimes)

['1979-01-02T00' '1979-01-03T00' '1979-01-04T00' ... '2019-12-28T00'
 '2019-12-29T00' '1900-01-01T00']


In [4]:
ix_end = np.where(dtimes == np.datetime64('2017-07-31T00'))[0][0]
print(ix_end)
pcp = pcp[:ix_end,:,:]
dtimes = dtimes[:ix_end]
times = times[:ix_end]

14090


In [5]:
#Print and check Variables
print(ncdf.variables.keys())
print(ncdf.dimensions)
print(ncdf.variables['pcp_Accum'])
print(ncdf.variables['lon'][:].min())
print(ncdf.variables['lat'][:].min())

ncdf.close()

print(np.shape(pcp))
PCP_max = np.max(pcp, axis=0)*1000
print(np.shape(PCP_max))
print(PCP_max)
np.percentile(PCP_max, 100)

dict_keys(['time', 'lon', 'lat', 'pcp_Accum'])
{'time': <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 14973, 'lon': <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 1440, 'lat': <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 121}
<class 'netCDF4._netCDF4.Variable'>
float32 pcp_Accum(time, lat, lon)
    units: m
unlimited dimensions: 
current shape = (14973, 121, 1440)
filling on, default _FillValue of 9.969209968386869e+36 used
-180.0
-90.0
(14090, 121, 1440)


<ipython-input-5-7aa3da2ae676>:4: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  print(ncdf.variables['pcp_Accum'])


(121, 1440)
[[32.563454  32.349197  30.541378  ... 34.3912    34.046272  33.656063 ]
 [36.69403   36.22645   34.54775   ... 36.431004  36.84023   36.95952  ]
 [35.83922   35.848663  35.9347    ... 35.66184   35.736362  35.82447  ]
 ...
 [ 4.4465046  4.450767   4.4542007 ...  4.4317045  4.4350195  4.4402294]
 [ 4.7627506  4.7660656  4.7684336 ...  4.755646   4.7589617  4.759435 ]
 [ 4.3142524  4.3142524  4.3142524 ...  4.3142524  4.3142524  4.3142524]]


110.35196685791016

In [6]:
#Calculate cumulative sum
PCP_c=pcp.cumsum(axis=0)
print(np.shape(PCP_c))


(14090, 121, 1440)


In [7]:
#Write numpy array as NetCDF
ncout = netCDF4.Dataset('ERA5_short_Accum.nc','w','NETCDF4_classic'); # using netCDF3 for output format 
ncout.createDimension('lon',np.size(lons))
ncout.createDimension('lat',np.size(lats))
ncout.createDimension('time',np.size(dtimes))

lonvar = ncout.createVariable('lon','float32',('lon')); lonvar[:] = lons;
latvar = ncout.createVariable('lat','float32',('lat')); latvar[:] = lats;
timevar = ncout.createVariable('time','float64',('time')); timevar[:] = times;
myvar = ncout.createVariable('pcp_accum','float32',('time','lat','lon')); myvar.setncattr('units','m'); myvar[:] = PCP_c;

ncout.close();

In [8]:
print(times)
print(dtimes)

[ 692520.  692544.  692568. ... 1030608. 1030632. 1030656.]
['1979-01-02T00' '1979-01-03T00' '1979-01-04T00' ... '2017-07-28T00'
 '2017-07-29T00' '2017-07-30T00']


In [9]:
ncdf.dimensions

RuntimeError: NetCDF: Not a valid ID

In [10]:
#Calculate datetime
dtimes = np.empty(len(times), dtype='datetime64[h]')
for i,t in enumerate(times):
    dtimes[i] = np.datetime64('1900-01-01') + np.timedelta64(np.int64(t),'h')
dtimes = dtimes.astype('datetime64')
dtimes

array(['1979-01-02T00', '1979-01-03T00', '1979-01-04T00', ...,
       '2017-07-28T00', '2017-07-29T00', '2017-07-30T00'],
      dtype='datetime64[h]')